In [7]:
# 调用模型，生成抗菌肽
import os
import time
from itertools import chain

import torch

from transformers import GPT2Tokenizer
from transformers import TextGenerationPipeline

In [8]:
tokenizer = GPT2Tokenizer('./vocab_file/vocab.json', './vocab_file/merges.txt')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

model_dir = '/home/xms/AMP-master/save_model/finetune-model/finetune_with_ADP3/finetune-10-64-30eGPT-50epochs'

model = torch.load(model_dir)

pipeline = TextGenerationPipeline(model, tokenizer)

In [3]:

greedy_search = pipeline('B', max_length=48, pad_token_id = 23) # greedy decoding
beam_search50 = pipeline('B', max_length=48, num_beams =50, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 10)
pure_sampling = pipeline('B', do_sample = True, max_length=48, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 10)
pure_samT05 = pipeline('B', do_sample = True, max_length=48, temperature = 0.5,repetitional_penalty=1.2, pad_token_id = 23, num_return_sequences = 10)
pure_samT07 = pipeline('B', do_sample = True, max_length=48, temperature = 0.7,repetitional_penalty=1.2, pad_token_id = 23, num_return_sequences = 10)
top_k3 = pipeline('B', do_sample = True, max_length=48, top_k = 3, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 50)
top_k5 = pipeline('B', do_sample = True, max_length=48, top_k = 5, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 50)
top_k10 = pipeline('B', do_sample = True, max_length=48, top_k = 10, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 100)
top_P05 = pipeline('B', do_sample = True, max_length=48, top_p = 0.5, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 50)
top_P07 = pipeline('B', do_sample = True, max_length=48, top_p = 0.7, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 50)
top_P09 = pipeline('B', do_sample = True, max_length=48, top_p = 0.9, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 50)
top_P095 = pipeline('B', do_sample = True, max_length=48, top_p = 0.95, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 50)


In [4]:
# 保存生成序列成fasta文件
import os
import time
from itertools import chain

full_list = list(tuple(chain(greedy_search, beam_search50, pure_sampling, pure_samT05, pure_samT07,
                 top_k3, top_k5, top_k10, top_P05, top_P07, top_P09, top_P095))) #合并列表
# print(full_list)
# 时间
current_time = int(time.time())
localtime = time.localtime(current_time)
datetime = time.strftime('%Y:%m:%d-%H:%M:%S', localtime)  # '%Y:%m:%d-%H:%M:%S' 年月日-时分秒
    
save_dir = '/xms/AMP-master/generate_file/generate_seq_file_2023/FT-ampbert-481/'
    
def generate_fasta_file():
    with open(save_dir+'P10-48-P30-F10--'+str(datetime), 'w') as f:
        for n, m in enumerate(full_list):
            for i, j in m.items():
                f.write('>generate_seq'+str(n+1)+ '\n')
                f.write(str(j)[1:]+'\n')


# generate_fasta_file()
    

In [9]:

top_k10_500 = pipeline('B', do_sample = True, max_length=48, top_k = 6, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 500)
# top_P095 = pipeline('B', do_sample = True, max_length=48, top_p = 0.95, pad_token_id = 23,repetitional_penalty=1.2, num_return_sequences = 500)

save_dir = '/home/xms/AMP-master/generate_file/generate_seq_file_2023/Generation_Strategies _comparison/'
current_time = int(time.time())
localtime = time.localtime(current_time)
datetime = time.strftime('%Y:%m:%d-%H:%M:%S', localtime)  # '%Y:%m:%d-%H:%M:%S' 年月日-时分秒

def generate_fasta_file():
    with open(save_dir+'top_k6-'+str(datetime), 'w') as f:
        for n, m in enumerate(top_k10_500):
            for i, j in m.items():
                f.write('>generate_seq'+str(n+1)+ '\n')
                f.write(str(j)[1:]+'\n')


generate_fasta_file()
                         

AttributeError: 'GPT2LMHeadModel' object has no attribute 'generation_config'